<a href="https://colab.research.google.com/github/raghavb-05/Yoga-Position-Classifier/blob/main/yoga_pose_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Preparing the Yoga Pose Image classification Dataset**
This notebook is for Image Classification using ResNet in PyTorch, where we trained a deep convolutional neural network to classify images from the Yoga Pose Image dataset with around 70% accuracy.

In [ ]:
!pip install opendatasets --upgrade --quiet

import opendatasets as od
 
dataset_url = 'https://www.kaggle.com/ashishjangra27/face-mask-12k-images-dataset'
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: varadkottawar
Your Kaggle Key: ··········


100%|██████████| 330M/330M [00:07<00:00, 46.3MB/s]


Below is the list of the names of the output classes in the dataset.
In our cases these are the names of the Yoga positions.

In [ ]:
import os
 
DATA_DIR = '/content/face-mask-12k-images-dataset/Face Mask Dataset/Test'
classes = os.listdir(DATA_DIR)
print(classes)

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [ ]:
import torchvision.transforms as T
img_size = 256
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

dataset = ImageFolder(DATA_DIR, T.Compose([T.Resize(img_size), 
                                           T.Pad(8, padding_mode='reflect'),
                                           T.RandomCrop(img_size), 
                                           T.ToTensor(), 
                                           T.Normalize(*imagenet_stats)]))

In [ ]:
import torch
random_seed = 42
torch.manual_seed(random_seed);

In [ ]:
from torch.utils.data.dataloader import DataLoader

batch_size=128

The dataset which we are using does not images with same shapes.
Our model requires images with same dimesnions. This code cell helps us in making dimensions same.
There are other functions present as well which increases the performance of our model.

In [ ]:
import torchvision.transforms as T

image_size = 256
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

dataset = ImageFolder(DATA_DIR)

In next cell we split our dataset in training and validation dataset.
Since this is a small dataset, we here use only 10% of our original dataset as validation dataset. Generally 40% is suggested.

In [ ]:
from torch.utils.data import random_split

val_pct = 0.1
val_size = int(val_pct * len(dataset))

train_ds, valid_ds = random_split(dataset, [len(dataset) - val_size, val_size])

Here we will be creating dataloaders that we will using for loading our dataset to the model

In [ ]:
from torch.utils.data import DataLoader

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(valid_ds, batch_size*2, num_workers=2, pin_memory=True)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from torchvision.utils import make_grid

def denormalize(images, means, stds):
    means = torch.tensor(means).reshape(1, 3, 1, 1)
    stds = torch.tensor(stds).reshape(1, 3, 1, 1)
    return images * stds + means

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        denorm_images = denormalize(images, *stats)
        ax.imshow(make_grid(denorm_images[:64], nrow=8).permute(1, 2, 0).clamp(0,1))
        break

Here are some of the images from our dataset.


In [ ]:
show_batch(train_dl)

TypeError: ignored

Next code cell helps us in calculating the accuracy and losses our model is creating.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels)  # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}],{} train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, "last_lr: {:.5f},".format(result['lrs'][-1]) if 'lrs' in result else '', 
            result['train_loss'], result['val_loss'], result['val_acc']))

#Modifying a Pretrained Model (ResNet34)
We are using a pretrained ResNet34 model here.
As the name itself suggest that the model is pretrained, which means that parameters already have some calculated values, our model just imporves it according to our dataset. 

In [ ]:
from torchvision import models

class YogaModel(ImageClassificationBase):
    def __init__(self, num_classes, pretrained=True):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet34(pretrained=pretrained)
        # Replace last layer
        self.network.fc = nn.Linear(self.network.fc.in_features, num_classes)

    def forward(self, xb):
        return self.network(xb)

#Using a GPU
To seamlessly use a GPU, if one is available, we define a couple of helper functions (get_default_device & to_device) and a helper class DeviceDataLoader to move our model & data to the GPU as required.

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""

    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

#Training the model
we are using techniques, that will increase the performance if our model, such as **Learning rate scheduling, Weight decay, Gradient clipping**
These are some articles that will help you in learning more about them

https://sgugger.github.io/the-1cycle-policy.html

https://towardsdatascience.com/this-thing-called-weight-decay-a7cd4bcfccab

https://towardsdatascience.com/what-is-gradient-clipping-b8e815cdfb48


In [ ]:
import torch
from tqdm.notebook import tqdm

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader,
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    # Set up custom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs,
                                                steps_per_epoch=len(train_loader))

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
device = get_default_device()
device

device(type='cuda')

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

#Creating our model

In [ ]:
model = YogaModel(len(classes))
to_device(model, device);

In [ ]:
history = [evaluate(model, val_dl)]
history

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


RuntimeError: ignored

Here we define our hyperparameters

In [ ]:
epochs = 10
max_lr = 0.002
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

We begin the traing of our model

In [ ]:
%%time
history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, 
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)

#**Predictions**
Here we will be testing our model by passing individual images and obtaining the predicted values.

In [ ]:
def predict_image(img, model):
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label
    return classes[preds[0].item()]

In [ ]:
img, label = valid_ds[105]
plt.imshow(img.permute(1, 2, 0))
print('Label:', classes[label], ', Predicted:', predict_image(img, model))

In [ ]:
torch.save(model.state_dict(), 'model')

In [ ]:
!pip install streamlit

import streamlit as st
st.write("""
         # Yoga Position Classifier
         """
         )
st.write("This is a simple image classification web app to classify yoga pose")
file = st.file_uploader("Please upload an image file", type=["jpg", "png"])

In [ ]:
import torch

In [ ]:
model = torch.load('/content/model')

In [ ]:
import cv2
from PIL import Image, ImageOps
import numpy as np
import torchvision.transforms as T

def predict_image(img, model):
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label
    return classes[preds[0].item()]

def import_and_predict(image_data, model):
    
        image_size = 256
        stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

        image = ImageFolder(image_data, transform=T.Compose([
        T.Resize(image_size),
        T.CenterCrop(image_size),
        T.RandomRotation(10),
        T.RandomResizedCrop(256, scale=(0.5,0.9), ratio=(1, 1)), 
        T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        T.ToTensor(),
        T.Normalize(*stats)]))
        print('Label:', classes[label], ', Predicted:', predict_image(img, model))

        return prediction
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    prediction = import_and_predict(image, model)